Cassandra:

Data Model Creation:
Design a data model for an e-commerce platform to handle products, orders, and user information.
Define appropriate column families and primary keys to ensure efficient querying.

Data Insertion and Retrieval:
Insert sample data into the Cassandra database, including user information and product details.
Retrieve a user's order history using CQL (Cassandra Query Language).

Time-Series Data:
Design a schema to handle time-series data, such as IoT sensor readings.
Insert and retrieve time-series data efficiently, using appropriate time-based partitioning.

Neo4j:

Graph Creation:
Build a graph database to represent a social network, with nodes for users and relationships for friendships.
Create nodes for users and relationships to model their interactions.

Querying Graph Data:
Write a Cypher query to find all friends of a given user up to a certain depth (degree of separation).
Retrieve common friends between two users in the graph.

Recommendation System:
Implement a basic recommendation system by finding users who have similar interests based on their interactions.
Propose movies, books, or other items that a user might be interested in based on their connections' preferences.

Pathfinding:
Use graph algorithms in Neo4j to find the shortest path between two nodes in the graph.
Find the path of mutual friends between two users.

In [3]:
from cassandra.cluster import Cluster
import uuid
from datetime import datetime
import time
import random

In [87]:
cluster = Cluster(['127.0.0.1'], port = 9042)
session = cluster.connect()

In [8]:
create_keyspace_query = """
CREATE KEYSPACE IF NOT EXISTS eshop
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
"""

In [9]:
session.execute(create_keyspace_query)

In [10]:
session.execute("USE eshop")

Data Model Creation:
Design a data model for an e-commerce platform to handle products, orders, and user information.
Define appropriate column families and primary keys to ensure efficient querying.

1. Products: 
product_id, product_name, description, price, category, amount

2. Users:
user_id, name, surname, email, age

3. Orders: 
order_id, user_id, product_id, order_date, amount_ordered, status

In [16]:
create_inventory_query = """
CREATE TABLE IF NOT EXISTS inventory(
product_id UUID PRIMARY KEY,
product_name TEXT,
description TEXT,
price DECIMAL,
category TEXT,
amount INT
)
"""
session.execute(create_inventory_query)

In [10]:
create_users_query = """
CREATE TABLE IF NOT EXISTS users(
user_id UUID PRIMARY KEY,
name TEXT,
surname TEXT,
age INT,
email TEXT
)
"""
session.execute(create_users_query)

In [11]:
create_orders_query = """
CREATE TABLE IF NOT EXISTS orders(
order_id UUID PRIMARY KEY,
user_id UUID,
product_id UUID,
order_date TIMESTAMP,
ordered_amount INT,
status TEXT
)
"""
session.execute(create_orders_query)

Data Insertion and Retrieval:
Insert sample data into the Cassandra database, including user information and product details.
Retrieve a user's order history using CQL (Cassandra Query Language).

In [19]:
insert_inventory_query = """INSERT INTO inventory (
    product_id,
    product_name,
    description,
    price,
    category,
    amount
    ) VALUES (?, ?, ?, ?, ?, ?)
    """

prepared_prod = session.prepare(insert_inventory_query)

session.execute(prepared_prod, (uuid.uuid4(), 'Pen 2000', 'Long-lasting blue pen', 0.69, 'Stationery', 2000))
session.execute(prepared_prod, (uuid.uuid4(), 'Computer Pear', 'Computer for gaming', 1999.99, 'Electronics', 10))
session.execute(prepared_prod, (uuid.uuid4(), 'Ethanol wipes', 'Wipes for computer, tablet, phone screens', 9.99, 'Supplies', 500))
session.execute(prepared_prod, (uuid.uuid4(), 'Post-its', 'Sticky notes', 1.99, 'Stationery', 700))
session.execute(prepared_prod, (uuid.uuid4(), 'Strawberry keyboard', 'Pink keyboard', 59.99, 'Electronics', 30))

In [20]:
insert_users_query = """INSERT INTO users (
    user_id,
    name,
    surname,
    age,
    email
    ) VALUES (?, ?, ?, ?, ?)
    """

prepared_user = session.prepare(insert_users_query)

session.execute(prepared_user, (uuid.uuid4(), 'Alice', 'Wonder', 19, 'alice@wonderr.com'))
session.execute(prepared_user, (uuid.uuid4(), 'Adam', 'Adams', 55, 'adamadams@ghgh.ixb'))
session.execute(prepared_user, (uuid.uuid4(), 'Evelyn', 'Evans', 27, 'evelynevans111@neibo.co'))
session.execute(prepared_user, (uuid.uuid4(), 'Alicia July', 'Roberts', 37, 'ajroberts@gamil.gom'))
session.execute(prepared_user, (uuid.uuid4(), 'Felix', 'Fernandez', 59, 'felixfernandez@yaho.cyc'))

In [21]:
test_query = session.execute("SELECT * FROM inventory WHERE product_name = 'Ethanol wipes' ALLOW FILTERING")

for row in test_query:
    print(row.product_id, row.product_name, row.amount)

8c17ea9c-683c-43d8-b73b-dd3460c7b366 Ethanol wipes 500


In [34]:
def order_now(email, product_name, amount):
    
    confirm_user_query = "SELECT * FROM users WHERE email = %s ALLOW FILTERING"
    confirm_user = session.execute(confirm_user_query, (email,))
    if not confirm_user:
        raise ValueError("Invalid user's email address")
    
    user_row = confirm_user.one()
    user_id = user_row.user_id
    
    confirm_product_query = "SELECT product_id, amount FROM inventory WHERE product_name = %s ALLOW FILTERING"
    confirm_product = session.execute(confirm_product_query, (product_name,))
    if not confirm_product:
        raise ValueError("Such product does not exist")
    
    product_row = confirm_product.one()
    product_id = product_row.product_id
    prod_amount = product_row.amount
    
    if amount > prod_amount:
        raise ValueError("Sorry, there is not enough of the requested product in stock")
    
    order_id = uuid.uuid4()
    order_date = datetime.now()
    status = "Ordered"
    
    insert_order_query = """
    INSERT INTO orders (
        order_id,
        user_id,
        product_id,
        order_date,
        ordered_amount,
        status
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    session.execute(
        insert_order_query,
        (order_id, user_id, product_id, order_date, amount, status)
    )


In [35]:
#ordered twice
try_order = order_now('alice@wonderr.com', 'Ethanol wipes', 20)

In [36]:
#ordered twice
one_more_order = order_now('evelynevans111@neibo.co', 'Ethanol wipes', 20)

In [40]:
uid = uuid.UUID('65c0b2fe-04f9-4f6b-94ab-40589924cece')

result = session.execute("SELECT * FROM orders WHERE user_id = {} ALLOW FILTERING".format(uid))
for row in result:
    print(row)

Row(order_id=UUID('4c7cb2c7-a767-4c83-80e8-d9c960dfd48c'), order_date=datetime.datetime(2023, 8, 20, 11, 55, 43, 130000), ordered_amount=20, product_id=UUID('8c17ea9c-683c-43d8-b73b-dd3460c7b366'), status='Ordered', user_id=UUID('65c0b2fe-04f9-4f6b-94ab-40589924cece'))
Row(order_id=UUID('c663227d-e8de-4207-9036-19a94553bfe7'), order_date=datetime.datetime(2023, 8, 20, 11, 45, 37, 606000), ordered_amount=20, product_id=UUID('8c17ea9c-683c-43d8-b73b-dd3460c7b366'), status='Ordered', user_id=UUID('65c0b2fe-04f9-4f6b-94ab-40589924cece'))


In [88]:
session.shutdown()
cluster.shutdown()

Time-Series Data:
Design a schema to handle time-series data, such as IoT sensor readings.
Insert and retrieve time-series data efficiently, using appropriate time-based partitioning.

In [4]:
cluster = Cluster(['127.0.0.1'], port = 9042)

In [43]:
create_keyspace_query_2 = """
CREATE KEYSPACE IF NOT EXISTS sensors
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
"""

In [44]:
session = cluster.connect()
session.execute(create_keyspace_query_2)

In [45]:
session.execute("USE sensors")

In [65]:
sensor_table_query = """
CREATE TABLE IF NOT EXISTS sensor_readings(
    room_id UUID,
    timestamp TIMESTAMP,
    temperature DECIMAL,
    PRIMARY KEY (room_id, timestamp)
) WITH CLUSTERING ORDER BY (timestamp DESC)
"""

session2.execute(sensor_table_query)

In [5]:
#creating rooms

all_rooms = []
for _ in range(100):
    letter_space = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E'}
    letter = letter_space[random.randint(1, 5)]
    number = random.randint(100, 999)
    room = letter + str(number)
    if room not in all_rooms:
        all_rooms.append(room)


In [90]:
rooms_table_query = """
CREATE TABLE IF NOT EXISTS roomss(
room_id UUID PRIMARY KEY,
room_name TEXT
)
"""
session2.execute(rooms_table_query)

In [10]:
for room in all_rooms:
    session2.execute(
        """
        INSERT INTO roomss (room_id, room_name)
        VALUES (%s, %s)
        """,
        (uuid.uuid4(), room)
        )

In [11]:
result = session2.execute("SELECT * FROM roomss")
for row in result:
    print(row)

Row(room_id=UUID('ce0a8d0f-697b-4ef2-b39d-fb8d94398bfa'), room_name='C548')
Row(room_id=UUID('2ccb4226-0442-431d-9723-7912955f5245'), room_name='D294')
Row(room_id=UUID('16f95da2-9d84-4720-a067-5c3f91407d5b'), room_name='B120')
Row(room_id=UUID('4bce85de-3b3b-4387-9a66-60bfddb7793a'), room_name='D957')
Row(room_id=UUID('5e082989-5122-4ef6-add1-d5bd36cec4df'), room_name='D296')
Row(room_id=UUID('2149bdd6-3b9e-4c7d-94fc-d7cbab22b2b0'), room_name='D227')
Row(room_id=UUID('cc4596f7-a074-4ff1-a6cb-047d413302d6'), room_name='A470')
Row(room_id=UUID('0dd19513-7d95-406a-95e7-18ee1fa7f30d'), room_name='D742')
Row(room_id=UUID('330a9f30-2f3e-427c-ae87-6a6dd3fefca3'), room_name='C609')
Row(room_id=UUID('8522bb72-fe66-4037-b0ca-2547eb0f7687'), room_name='B417')
Row(room_id=UUID('4f73488f-6b7b-426d-8a08-809d22f55df6'), room_name='D705')
Row(room_id=UUID('c6b96123-2adc-466a-906a-81f7467e7dfa'), room_name='E177')
Row(room_id=UUID('c358295e-92b5-4c3f-9d2d-768b2ce9ce9f'), room_name='E270')
Row(room_id=

In [43]:
#creating data for sensors
for _ in range(1000):
    room_index = random.randint(0, 99)
    room_name = all_rooms[room_index]
    room_query = """SELECT room_id FROM roomss 
    WHERE room_name = %s ALLOW FILTERING"""
    room_execute = session2.execute(room_query, (room_name,)).one()
    room_id = room_execute.room_id
    temp = 22.0 + random.uniform(-6.0, 4.0)
    insert_sensor_query = """
    INSERT INTO sensor_readings (room_id, timestamp, temperature)
    VALUES (%s, %s, %s)
    """
    session2.execute(insert_sensor_query, (room_id, datetime.now(), temp))
    time.sleep(2)

In [44]:
check = session2.execute("SELECT * FROM sensor_readings")
for i in check:
    print(i)

Row(room_id=UUID('ce0a8d0f-697b-4ef2-b39d-fb8d94398bfa'), timestamp=datetime.datetime(2023, 8, 20, 15, 4, 1, 746000), temperature=Decimal('17.375071288399816'))
Row(room_id=UUID('ce0a8d0f-697b-4ef2-b39d-fb8d94398bfa'), timestamp=datetime.datetime(2023, 8, 20, 15, 1, 47, 38000), temperature=Decimal('23.346784619999102'))
Row(room_id=UUID('ce0a8d0f-697b-4ef2-b39d-fb8d94398bfa'), timestamp=datetime.datetime(2023, 8, 20, 15, 0, 22, 585000), temperature=Decimal('20.586166355231505'))
Row(room_id=UUID('ce0a8d0f-697b-4ef2-b39d-fb8d94398bfa'), timestamp=datetime.datetime(2023, 8, 20, 14, 59, 20, 254000), temperature=Decimal('23.054654199921533'))
Row(room_id=UUID('ce0a8d0f-697b-4ef2-b39d-fb8d94398bfa'), timestamp=datetime.datetime(2023, 8, 20, 14, 58, 3, 870000), temperature=Decimal('23.29993185949646'))
Row(room_id=UUID('ce0a8d0f-697b-4ef2-b39d-fb8d94398bfa'), timestamp=datetime.datetime(2023, 8, 20, 14, 58, 1, 861000), temperature=Decimal('22.711437369950453'))
Row(room_id=UUID('ce0a8d0f-697

In [49]:
start_time = datetime(2023, 8, 20, 14, 30, 21, 419000)
end_time = datetime.now()

select_ontime = """
SELECT * FROM sensor_readings WHERE room_id = %s AND timestamp >= %s AND timestamp <= %s
ALLOW FILTERING"""

select_room_id = """
SELECT room_id FROM roomss WHERE room_name = %s ALLOW FILTERING
"""
roomid = session2.execute(select_room_id, (all_rooms[55],)).one()
ids = roomid.room_id

results = session2.execute(select_ontime, (ids, start_time, end_time))
for row in results:
    print(row)

Row(room_id=UUID('e46cbdac-04a0-4ce8-b668-8d93a0d7ecc9'), timestamp=datetime.datetime(2023, 8, 20, 15, 5, 2, 36000), temperature=Decimal('24.521195451868266'))
Row(room_id=UUID('e46cbdac-04a0-4ce8-b668-8d93a0d7ecc9'), timestamp=datetime.datetime(2023, 8, 20, 15, 3, 49, 683000), temperature=Decimal('19.64189490487839'))
Row(room_id=UUID('e46cbdac-04a0-4ce8-b668-8d93a0d7ecc9'), timestamp=datetime.datetime(2023, 8, 20, 14, 56, 11, 296000), temperature=Decimal('19.647352956482642'))
Row(room_id=UUID('e46cbdac-04a0-4ce8-b668-8d93a0d7ecc9'), timestamp=datetime.datetime(2023, 8, 20, 14, 55, 33, 97000), temperature=Decimal('16.877651588403417'))
Row(room_id=UUID('e46cbdac-04a0-4ce8-b668-8d93a0d7ecc9'), timestamp=datetime.datetime(2023, 8, 20, 14, 54, 52, 887000), temperature=Decimal('21.44490574594525'))
Row(room_id=UUID('e46cbdac-04a0-4ce8-b668-8d93a0d7ecc9'), timestamp=datetime.datetime(2023, 8, 20, 14, 52, 18, 49000), temperature=Decimal('23.384387876201316'))
Row(room_id=UUID('e46cbdac-04a

Neo4j:

Graph Creation:
Build a graph database to represent a social network, with nodes for users and relationships for friendships.
Create nodes for users and relationships to model their interactions.

Querying Graph Data:
Write a Cypher query to find all friends of a given user up to a certain depth (degree of separation).
Retrieve common friends between two users in the graph.

Recommendation System:
Implement a basic recommendation system by finding users who have similar interests based on their interactions.
Propose movies, books, or other items that a user might be interested in based on their connections' preferences.

Pathfinding:
Use graph algorithms in Neo4j to find the shortest path between two nodes in the graph.
Find the path of mutual friends between two users.

In [1]:
from neo4j import GraphDatabase

In [2]:
uri = "neo4j+ssc://890c3246.databases.neo4j.io"
username = "neo4j"
password = None #use appropriate database and password

In [3]:
driver = GraphDatabase.driver(uri,auth=(username,password))

Graph Creation:
Build a graph database to represent a social network, with nodes for users and relationships for friendships.
Create nodes for users and relationships to model their interactions.


In [8]:
def create_user(tx, user_id, name, age):
    query = "CREATE (u:User {id: $user_id, name: $name, age: $age})"
    tx.run(query, user_id=user_id, name=name, age=age)

In [13]:
with driver.session() as session:
    session.execute_write(create_user, 1, "Ieva", 26)
    session.execute_write(create_user, 2, "Estella", 33)
    session.execute_write(create_user, 3, "Jean", 18)
    session.execute_write(create_user, 4, "Monika", 22)
    session.execute_write(create_user, 5, "Justina", 45)
    session.execute_write(create_user, 6, "Laimonas", 62)
    session.execute_write(create_user, 7, "Valium", 19)
    session.execute_write(create_user, 8, "Martin", 29)

In [15]:
def create_event(tx, event_id, name):
    query = "CREATE (:Event {id: $event_id, name: $name})"
    tx.run(query, event_id=event_id, name=name)

In [16]:
with driver.session() as session:
    session.execute_write(create_event, 101, "Bikers convention")
    session.execute_write(create_event, 102, "Origami lecture")
    session.execute_write(create_event, 103, "Donation evening")

In [5]:
def create_friendship(tx, user_id1, user_id2):
    query = "MATCH (u1:User {id: $user_id1}), (u2:User {id: $user_id2}) " \
            "CREATE (u1)-[:FRIEND]->(u2)"
    tx.run(query, user_id1=user_id1, user_id2=user_id2)

In [14]:
with driver.session() as session:
    session.execute_write(create_friendship, 1, 2)
    session.execute_write(create_friendship, 3, 4)
    session.execute_write(create_friendship, 5, 6)
    session.execute_write(create_friendship, 7, 8)
    session.execute_write(create_friendship, 2, 5)
    session.execute_write(create_friendship, 8, 3)
    session.execute_write(create_friendship, 7, 1)
    session.execute_write(create_friendship, 5, 3)
    session.execute_write(create_friendship, 1, 8)

In [43]:
with driver.session() as session:
    session.execute_write(create_friendship, 1, 5)
    session.execute_write(create_friendship, 3, 7)
    session.execute_write(create_friendship, 1, 6)
    session.execute_write(create_friendship, 7, 4)
    session.execute_write(create_friendship, 4, 5)

In [18]:
def like_event(tx, user_id, event_id):
    query = "MATCH (u:User {id: $user_id}), (e:Event {id: $event_id}) " \
            "CREATE (u)-[:LIKES]->(e)"
    tx.run(query, user_id=user_id, event_id=event_id)

In [19]:
with driver.session() as session:
    session.execute_write(like_event, 1, 101)
    session.execute_write(like_event, 2, 102)
    session.execute_write(like_event, 3, 103)
    session.execute_write(like_event, 4, 101)
    session.execute_write(like_event, 5, 102)
    session.execute_write(like_event, 5, 103)
    session.execute_write(like_event, 6, 101)
    session.execute_write(like_event, 7, 102)
    session.execute_write(like_event, 8, 103)

Querying Graph Data:
Write a Cypher query to find all friends of a given user up to a certain depth (degree of separation).
Retrieve common friends between two users in the graph.

In [21]:
def find_friends(user_name, depth):
    query = (
        "MATCH (user:User {name: $user_name})-[:FRIEND*1.." + str(depth) + "]-(friend) "
        "RETURN friend.name AS friend_name"
    )
    with driver.session() as session:
        result = session.run(query, user_name=user_name)
        return [record["friend_name"] for record in result]


In [23]:
ieva_friends = find_friends("Ieva", 1)
print(ieva_friends)
ieva_friends_of_friends = find_friends("Ieva", 2)
print(ieva_friends_of_friends)

['Martin', 'Valium', 'Estella']
['Martin', 'Jean', 'Valium', 'Valium', 'Martin', 'Estella', 'Justina']


In [24]:
def find_common_friends(user1_name, user2_name):
    query = (
        "MATCH (u1:User {name: $user1_name})-[:FRIEND]-(common:User)-[:FRIEND]-(u2:User {name: $user2_name}) "
        "RETURN common.name AS common_friend"
    )
    with driver.session() as session:
        result = session.run(query, user1_name=user1_name, user2_name=user2_name)
        return [record["common_friend"] for record in result]


In [27]:
common_friends = find_common_friends("Ieva", "Martin")
print(common_friends)

['Valium']


Recommendation System:
Implement a basic recommendation system by finding users who have similar interests based on their interactions.
Propose movies, books, or other items that a user might be interested in based on their connections' preferences.

In [53]:
def find_similar_users(user_id):
    query = (
        "MATCH (user1:User {id: $user_id})-[:LIKES]->(Event)<-[:LIKES]-(user2:User) "
        "WHERE user1 <> user2 "
        "RETURN user2.name AS similar_user, COLLECT(Event.name) AS common_interests"
    )
    with driver.session() as session:
        result = session.run(query, user_id=user_id)
        return [{"user": record["similar_user"], "common_interests": record["common_interests"]} for record in result]

In [58]:
def recommend_event(user_id):
    query = (
        "MATCH (user:User {id: $user_id})-[:LIKES]->(:Event)<-[:LIKES]-(connection:User)-[:LIKES]->(event:Event) "
        "WHERE NOT (user)-[:LIKES]->(event) "
        "RETURN DISTINCT event.name AS suggested_event"
    )
    with driver.session() as session:
        result = session.run(query, user_id=user_id)
        return [record["suggested_event"] for record in result]

In [55]:
try_similar_user = find_similar_users(1)
print(try_similar_user)

[{'user': 'Laimonas', 'common_interests': ['Bikers convention']}, {'user': 'Monika', 'common_interests': ['Bikers convention']}]


In [60]:
try_event = recommend_event(2)
print(try_event)

['Donation evening']


Pathfinding:
Use graph algorithms in Neo4j to find the shortest path between two nodes in the graph.
Find the path of mutual friends between two users.

In [83]:
def find_shortest_path(transaction,id1,id2):
    result = transaction.run("MATCH (u1:User {id:$id1}),(u2:User {id:$id2}), path= SHORTESTPATH ((u1)-[*]->(u2)) RETURN nodes(path),relationships(path)",id1=id1,id2=id2)
    return [{"nodes":record["nodes(path)"],"relationships":record["relationships(path)"]} for record in result]

In [84]:
with driver.session() as session:
    results = session.execute_read(find_shortest_path,1,4)
    nodes = results[0]["nodes"]
    relationships = results[0]["relationships"]

In [85]:
for node in nodes:
    print(node.get("name"))

Ieva
Justina
Jean
Monika


In [87]:
for relationship in relationships:
    nodes = relationship.nodes
    print(nodes[0].get("name"),"->",nodes[1].get("name"))

Ieva -> Justina
Justina -> Jean
Jean -> Monika


In [88]:
def find_mutual_friends(id1, id2):
    query = (
        "MATCH path = shortestPath((u1:User {id: $id1})-[:FRIEND*]-(u2:User {id: $id2})) "
        "WHERE length(path) > 2 "
        "RETURN [user IN nodes(path) | user.name] AS mutual_friends"
    )
    with driver.session() as session:
        result = session.run(query, id1=id1, id2=id2)
        return result.single()["mutual_friends"]

In [89]:
mutual_friends = find_mutual_friends(2, 6)
print(mutual_friends)

['Estella', 'Justina', 'Ieva', 'Laimonas']
